In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import secrets
import re

In [2]:
url = "https://www.billboard.com/charts/year-end/2019/hot-100-songs"
billboard_page = requests.get(url)
soup = BeautifulSoup(billboard_page.text, "html.parser")
titles = [div.text for div in soup.find_all("div", "ye-chart-item__title")]
ranks = [div.text for div in soup.find_all("div", "ye-chart-item__rank")]
artists = [div.text for div in soup.find_all("div", "ye-chart-item__artist")]

top_songs = []
for i in range(0, len(ranks)):
    top_songs.append({"title": titles[i].replace("\n", ""), "artist": artists[i].replace("\n", "")})
    print(top_songs[i])

{'title': 'Old Town Road', 'artist': 'Lil Nas X Featuring Billy Ray Cyrus'}
{'title': 'Sunflower (Spider-Man: Into The Spider-Verse)', 'artist': 'Post Malone & Swae Lee'}
{'title': 'Without Me', 'artist': 'Halsey'}
{'title': 'Bad Guy', 'artist': 'Billie Eilish'}
{'title': 'Wow.', 'artist': 'Post Malone'}
{'title': 'Happier', 'artist': 'Marshmello & Bastille'}
{'title': '7 Rings', 'artist': 'Ariana Grande'}
{'title': 'Talk', 'artist': 'Khalid'}
{'title': 'Sicko Mode', 'artist': 'Travis Scott'}
{'title': 'Sucker', 'artist': 'Jonas Brothers'}
{'title': 'High Hopes', 'artist': 'Panic! At The Disco'}
{'title': 'Thank U, Next', 'artist': 'Ariana Grande'}
{'title': 'Truth Hurts', 'artist': 'Lizzo'}
{'title': 'Dancing With A Stranger', 'artist': 'Sam Smith & Normani'}
{'title': 'Senorita', 'artist': 'Shawn Mendes & Camila Cabello'}
{'title': "I Don't Care", 'artist': 'Ed Sheeran & Justin Bieber'}
{'title': 'Eastside', 'artist': 'benny blanco, Halsey & Khalid'}
{'title': 'Going Bad', 'artist': 

In [3]:
def request_song_info(song_title, artist_name):
    base_url = 'https://api.genius.com'
    headers = {'Authorization': 'Bearer ' + secrets.GENIUS_ACCESS_TOKEN}
    search_url = base_url + '/search'
    data = {'q': song_title + ' ' + artist_name}
    response = requests.get(search_url, data=data, headers=headers)

    return response

In [12]:
def is_artists_matching(billboard, genius_hit):
    secondary_artist = []
    featuring_split = []
    primary_artists_billboard = []
    billboard = billboard.lower()
    
    if("featuring" in billboard):
        featuring_split=re.split("featuring ", billboard)
        secondary_artist = featuring_split[1:]
        primary_artists_billboard = re.split("& |, ", featuring_split[0])
    elif("ft." in billboard):
        featuring_split=re.split("ft. ", billboard)
        secondary_artist = featuring_split[1:]
        primary_artists_billboard = re.split("& |, ", featuring_split[0])
    elif('feat.' in billboard):
        featuring_split=re.split("feat. ", billboard)
        secondary_artist = featuring_split[1:]
        primary_artists_billboard = re.split("& |, ", featuring_split[0])
    else:
        primary_artists_billboard = re.split("& |, ", billboard)
    contains_all_artists = True
    print("primary" + str(primary_artists_billboard))
    print("secondary" + str(secondary_artist))
    
    for artist in primary_artists_billboard:
        if(artist.lower().strip() not in genius_hit['result']['primary_artist']['name'].lower()):
            print("primary artist not found")
            return False
    
#     if(not secondary_artist and secondary_artist.lower() not in genius_hit['result']['title_with_featured'].lower()):
    if(secondary_artist):
        for second in secondary_artist:
            if second.lower().strip() not in genius_hit['result']['full_title'].lower():
                print(second.lower().strip() +" -- "+ genius_hit['result']['full_title'].lower())
                print("secondary artist not found")
                return False
    
    return contains_all_artists

In [13]:
def get_best_lyrics(song, response_hits):
    lyric_result  = "https://www.genius.com"
    for hit in response_hits:
#         print("info: "+ song['artist'].lower())
        if(is_artists_matching(song['artist'].lower(), hit )):
            return lyric_result + hit['result']['path']
    return  "not found"

In [14]:
def remove_para(title):
    song = list(title)
    newtitle = ""
    in_para = False
    for char in song:
        if char == "(":
            in_para = True
            continue
        elif char == ")":
            in_para = False
        
        if not in_para and char != ")":
            newtitle = newtitle + char
    return newtitle

def get_primary_artist(artist):
    secondary_artist = ""
    featuring_split = []
    if("featuring" in artist):
        featuring_split=re.split("featuring ", artist)
        secondary_artist = featuring_split[1:]
        primary_artists_billboard = re.split("& |, ", featuring_split[0])
    elif("ft." in artist):
        featuring_split=re.split("ft. ", artist)
        secondary_artist = featuring_split[1:]
        primary_artists_billboard = re.split("& |, ", featuring_split[0])
    elif('feat.' in artist):
        featuring_split=re.split("feat. ", artist)
        secondary_artist = featuring_split[1:]
        primary_artists_billboard = re.split("& |, ", featuring_split[0])       
    elif(' x ' in artist):
        featuring_split=re.split(" x ", artist)
        secondary_artist = featuring_split[1:]
        primary_artists_billboard = re.split("& |, ", featuring_split[0])
    else:
         primary_artists_billboard = re.split("& |, ", artist)
    return primary_artists_billboard[0]

In [15]:
for song in top_songs:
    print("--> searching: " +remove_para(song["title"])+ " "+ song["artist"].lower())
    response = request_song_info(remove_para(song["title"]), song["artist"].lower())
    json = response.json()
    lyric_url = get_best_lyrics(song, json['response']['hits'])
    if(lyric_url == "not found"): #try with just primary artist

        print("--> searching again: " +remove_para(song["title"]), get_primary_artist(song["artist"].lower()))
        response = request_song_info(remove_para(song["title"]), get_primary_artist(song["artist"].lower()))
        json = response.json()
        lyric_url = get_best_lyrics(song, json['response']['hits'])
    print(lyric_url)

--> searching: Old Town Road lil nas x featuring billy ray cyrus
primary['lil nas x ']
secondary['billy ray cyrus']
https://www.genius.com/Lil-nas-x-billy-ray-cyrus-and-diplo-old-town-road-diplo-remix-lyrics
--> searching: Sunflower  post malone & swae lee
primary['post malone ', 'swae lee']
secondary[]
https://www.genius.com/Post-malone-and-swae-lee-sunflower-lyrics
--> searching: Without Me halsey
primary['halsey']
secondary[]
https://www.genius.com/Halsey-without-me-lyrics
--> searching: Bad Guy billie eilish
primary['billie eilish']
secondary[]
https://www.genius.com/Billie-eilish-bad-guy-lyrics
--> searching: Wow. post malone
primary['post malone']
secondary[]
https://www.genius.com/Post-malone-wow-lyrics
--> searching: Happier marshmello & bastille
primary['marshmello ', 'bastille']
secondary[]
https://www.genius.com/Marshmello-and-bastille-happier-lyrics
--> searching: 7 Rings ariana grande
primary['ariana grande']
secondary[]
https://www.genius.com/Ariana-grande-7-rings-lyrics


primary['post malone']
secondary[]
https://www.genius.com/Post-malone-better-now-lyrics
--> searching: Youngblood 5 seconds of summer
primary['5 seconds of summer']
secondary[]
https://www.genius.com/5-seconds-of-summer-youngblood-lyrics
--> searching: Money In The Grave drake featuring rick ross
primary['drake ']
secondary['rick ross']
primary artist not found
primary['drake ']
secondary['rick ross']
primary artist not found
primary['drake ']
secondary['rick ross']
primary artist not found
primary['drake ']
secondary['rick ross']
primary artist not found
primary['drake ']
secondary['rick ross']
primary artist not found
primary['drake ']
secondary['rick ross']
primary artist not found
primary['drake ']
secondary['rick ross']
primary artist not found
primary['drake ']
secondary['rick ross']
primary artist not found
primary['drake ']
secondary['rick ross']
rick ross -- money in the grave by drake (ft. rick ross)
secondary artist not found
primary['drake ']
secondary['rick ross']
primary 

primary['morgan wallen']
secondary[]
https://www.genius.com/Morgan-wallen-whiskey-glasses-lyrics
--> searching: God's Country blake shelton
primary['blake shelton']
secondary[]
https://www.genius.com/Blake-shelton-gods-country-lyrics
--> searching: Be Alright dean lewis
primary['dean lewis']
secondary[]
https://www.genius.com/Dean-lewis-be-alright-lyrics
--> searching: Pure Water mustard & migos
primary['mustard ', 'migos']
secondary[]
https://www.genius.com/Mustard-and-migos-pure-water-lyrics
--> searching: The Git Up blanco brown
primary['blanco brown']
secondary[]
https://www.genius.com/Blanco-brown-the-git-up-lyrics
--> searching: Taki Taki dj snake featuring selena gomez, ozuna & cardi b
primary['dj snake ']
secondary['selena gomez, ozuna & cardi b']
primary artist not found
primary['dj snake ']
secondary['selena gomez, ozuna & cardi b']
primary artist not found
primary['dj snake ']
secondary['selena gomez, ozuna & cardi b']
primary artist not found
primary['dj snake ']
secondary[

primary['jonas brothers']
secondary[]
https://www.genius.com/Jonas-brothers-only-human-lyrics
--> searching: Knockin' Boots  luke bryan
primary[' luke bryan']
secondary[]
https://www.genius.com/Luke-bryan-knockin-boots-lyrics
--> searching: Trip ella mai
primary['ella mai']
secondary[]
https://www.genius.com/Ella-mai-trip-lyrics
--> searching: Rumor lee brice
primary['lee brice']
secondary[]
https://www.genius.com/Lee-brice-rumor-lyrics
--> searching: Swervin a boogie wit da hoodie featuring 6ix9ine
primary['a boogie wit da hoodie ']
secondary['6ix9ine']
https://www.genius.com/A-boogie-wit-da-hoodie-swervin-lyrics
--> searching: How Do You Sleep? sam smith
primary['sam smith']
secondary[]
https://www.genius.com/Sam-smith-how-do-you-sleep-lyrics
--> searching: Baby lil baby & dababy
primary['lil baby ', 'dababy']
secondary[]
primary artist not found
primary['lil baby ', 'dababy']
secondary[]
https://www.genius.com/Quality-control-lil-baby-and-dababy-baby-lyrics
--> searching: Look What 

primary['juice wrld']
secondary[]
https://www.genius.com/Juice-wrld-robbery-lyrics
